In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Hilo,US,2022-02-06 00:15:35,19.7297,-155.0900,78.53,78,100,9.22
5,5,Kapaa,US,2022-02-06 00:17:00,22.0752,-159.3190,80.19,65,0,1.99
10,10,Geraldton,AU,2022-02-06 00:17:02,-28.7667,114.6000,77.41,78,100,12.66
11,11,Roebourne,AU,2022-02-06 00:17:02,-20.7833,117.1333,86.18,68,55,9.37
12,12,Alyangula,AU,2022-02-06 00:17:03,-13.8483,136.4192,83.17,68,78,7.76
14,14,Teshie,GH,2022-02-06 00:17:03,5.5836,-0.1072,82.96,83,0,3.44
15,15,Butaritari,KI,2022-02-06 00:15:39,3.0707,172.7902,82.67,74,51,18.70
17,17,Xai-Xai,MZ,2022-02-06 00:17:04,-25.0519,33.6442,76.03,80,13,14.12
21,21,Kieta,PG,2022-02-06 00:14:16,-6.2167,155.6333,86.65,67,56,8.99
26,26,Bowen,AU,2022-02-06 00:17:08,-20.0167,148.2333,81.48,62,5,16.37


In [11]:
preferred_cities_df.count()

City_ID       147
City          147
Country       147
Date          147
Lat           147
Lng           147
Max Temp      147
Humidity      147
Cloudiness    147
Wind Speed    147
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,78.53,19.7297,-155.0900,
5,Kapaa,US,80.19,22.0752,-159.3190,
10,Geraldton,AU,77.41,-28.7667,114.6000,
11,Roebourne,AU,86.18,-20.7833,117.1333,
12,Alyangula,AU,83.17,-13.8483,136.4192,
14,Teshie,GH,82.96,5.5836,-0.1072,
15,Butaritari,KI,82.67,3.0707,172.7902,
17,Xai-Xai,MZ,76.03,-25.0519,33.6442,
21,Kieta,PG,86.65,-6.2167,155.6333,
26,Bowen,AU,81.48,-20.0167,148.2333,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,78.53,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Kapaa,US,80.19,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Geraldton,AU,77.41,-28.7667,114.6000,Broadwater Mariner Resort
11,Roebourne,AU,86.18,-20.7833,117.1333,Latitude20 Roebourne Village
12,Alyangula,AU,83.17,-13.8483,136.4192,Groote Eylandt Lodge
14,Teshie,GH,82.96,5.5836,-0.1072,Best Western Plus Accra Beach Hotel
15,Butaritari,KI,82.67,3.0707,172.7902,Isles Sunset Lodge
17,Xai-Xai,MZ,76.03,-25.0519,33.6442,Complexo da Familia
21,Kieta,PG,86.65,-6.2167,155.6333,Uruna Bay Retreat
26,Bowen,AU,81.48,-20.0167,148.2333,Castle Motor Lodge Motel


In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

NameError: name 'hotel_info' is not defined